In [1]:
#import library and read input data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.colorbar import ColorbarBase

from pandas import set_option
set_option("display.max_rows", 10)
filename = 'facies_vectors.csv'
training_data = pd.read_csv(filename)

In [2]:
training_data.describe()

,Facies,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
count,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,4149.000000,3232.000000,4149.000000,4149.000000
mean,4.503254,2906.867438,64.933985,0.659566,4.402484,13.201066,3.725014,1.518438,0.521852
std,2.474324,133.300164,30.302530,0.252703,5.274947,7.132846,0.896152,0.499720,0.286644
min,1.000000,2573.500000,10.149000,-0.025949,-21.832000,0.550000,0.200000,1.000000,0.000000
25%,2.000000,2821.500000,44.730000,0.498000,1.600000,8.500000,3.100000,1.000000,0.277000
50%,4.000000,2932.500000,64.990000,0.639000,4.300000,12.020000,3.551500,2.000000,0.528000
75%,6.000000,3007.000000,79.438000,0.822000,7.500000,16.050000,4.300000,2.000000,0.769000
max,9.000000,3138.000000,361.150000,1.800000,19.312000,84.400000,8.094000,2.000000,1.000000


In [3]:
training_data.head()

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915


In [4]:
training_data.columns


Index(['Facies', 'Formation', 'Well Name', 'Depth', 'GR', 'ILD_log10',
       'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS'],
      dtype='object')

In [5]:
training_data["Well Name"].unique()

array(['SHRIMPLIN', 'ALEXANDER D', 'SHANKLE', 'LUKE G U', 'KIMZEY A',
       'CROSS H CATTLE', 'NOLAN', 'Recruit F9', 'NEWBY',
       'CHURCHMAN BIBLE'], dtype=object)

In [6]:
blind = training_data[training_data['Well Name'] == 'CHURCHMAN BIBLE']
training_data = training_data[training_data['Well Name'] != 'CHURCHMAN BIBLE']

# clean up data and change Well Name and formation as categorial data

training_data['Well Name'] = training_data['Well Name'].astype('category')
training_data['Formation'] = training_data['Formation'].astype('category')
training_data['Well Name'].unique()

['SHRIMPLIN', 'ALEXANDER D', 'SHANKLE', 'LUKE G U', 'KIMZEY A', 'CROSS H CATTLE', 'NOLAN', 'Recruit F9', 'NEWBY']
Categories (9, object): ['ALEXANDER D', 'CROSS H CATTLE', 'KIMZEY A', 'LUKE G U', ..., 'NOLAN', 'Recruit F9', 'SHANKLE', 'SHRIMPLIN']

In [7]:
blind['Well Name'] = blind['Well Name'].astype('category')
blind.head()

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
3745,3,A1 SH,CHURCHMAN BIBLE,2917.5,63.500,0.705,4.916,12.174,3.162,1,1.00
3746,3,A1 SH,CHURCHMAN BIBLE,2918.0,80.875,0.709,7.514,13.286,2.932,1,0.95
3747,3,A1 SH,CHURCHMAN BIBLE,2918.5,87.813,0.700,7.785,13.212,2.953,1,0.90
3748,3,A1 SH,CHURCHMAN BIBLE,2919.0,86.750,0.685,7.547,13.547,3.420,1,0.85
3749,3,A1 SH,CHURCHMAN BIBLE,2919.5,79.438,0.669,5.912,12.252,3.324,1,0.80


# Missing PE well


In [10]:
training_data.isna().sum()

Facies         0
Formation      0
Well Name      0
Depth          0
GR             0
            ... 
DeltaPHI       0
PHIND          0
PE           917
NM_M           0
RELPOS         0
Length: 11, dtype: int64

In [8]:
# Check for missing PE values
wells_missing_PE = training_data[training_data['PE'].isnull()]

# Display unique well names with missing PE
print("Wells with missing PE:", wells_missing_PE['Well Name'].unique())

# Save these rows to a new CSV file
wells_missing_PE.to_csv('wells_missing_PE.csv', index=False)


Wells with missing PE: ['ALEXANDER D', 'KIMZEY A', 'Recruit F9']
Categories (9, object): ['ALEXANDER D', 'CROSS H CATTLE', 'KIMZEY A', 'LUKE G U', ..., 'NOLAN', 'Recruit F9', 'SHANKLE', 'SHRIMPLIN']


In [11]:
from sklearn.model_selection import train_test_split

# Remove wells with missing PE
remaining_data = training_data.dropna(subset=['PE'])

# Get unique wells
unique_wells = remaining_data['Well Name'].unique()

# Split wells into train and test (e.g., 80% train, 20% test)
train_wells, test_wells = train_test_split(unique_wells, test_size=0.2, random_state=42)

# Filter data based on the split
train_data = remaining_data[remaining_data['Well Name'].isin(train_wells)]
test_data = remaining_data[remaining_data['Well Name'].isin(test_wells)]

# Optional: check the distribution
print("Training wells:", train_wells)
print("Test wells:", test_wells)
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

# Save train and test datasets
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)


Training wells: ['Recruit F9', 'LUKE G U', 'NOLAN', 'CROSS H CATTLE', 'NEWBY']
Categories (9, object): ['ALEXANDER D', 'CROSS H CATTLE', 'KIMZEY A', 'LUKE G U', ..., 'NOLAN', 'Recruit F9', 'SHANKLE', 'SHRIMPLIN']
Test wells: ['SHRIMPLIN', 'SHANKLE']
Categories (9, object): ['ALEXANDER D', 'CROSS H CATTLE', 'KIMZEY A', 'LUKE G U', ..., 'NOLAN', 'Recruit F9', 'SHANKLE', 'SHRIMPLIN']
Train data shape: (1908, 11)
Test data shape: (920, 11)


In [17]:
# -----------------------------
# Import Libraries
# -----------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder

# -----------------------------
# Prepare Data
# -----------------------------
# Drop rows with missing PE for training
train_data_ml = train_data.dropna(subset=['PE']).copy()

# Use OrdinalEncoder for categorical features with unseen handling
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
categorical_cols = ['Well Name', 'Formation']

train_data_ml[categorical_cols] = encoder.fit_transform(train_data_ml[categorical_cols])

# Features and target
X_train = train_data_ml.drop(columns=['PE'])
y_train = train_data_ml['PE']

# Prepare test data
test_data_ml = test_data.copy()
test_data_ml[categorical_cols] = encoder.transform(test_data_ml[categorical_cols])
X_test = test_data_ml.drop(columns=['PE'])
y_test = test_data_ml['PE']

# -----------------------------
# Train Random Forest Regressor
# -----------------------------
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred = rf_model.predict(X_test)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Test MSE: {mse:.4f}, R2: {r2:.4f}")

# -----------------------------
# Predict missing PE
# -----------------------------
missing_pe_data = training_data[training_data['PE'].isnull()].copy()
missing_pe_data[categorical_cols] = encoder.transform(missing_pe_data[categorical_cols])
X_missing = missing_pe_data.drop(columns=['PE'])
missing_pe_data['PE'] = rf_model.predict(X_missing)

# Save predicted PE
missing_pe_data.to_csv('predicted_PE.csv', index=False)

# -----------------------------
# Plot Predicted vs Actual
# -----------------------------
plt.figure(figsize=(8,6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual PE')
plt.ylabel('Predicted PE')
plt.title('Predicted vs Actual PE (Test Set)')
plt.grid(True)
plt.show()

# Distribution of predicted PE for missing values
plt.figure(figsize=(10,6))
sns.histplot(missing_pe_data['PE'], kde=True, bins=30, color='skyblue')
plt.xlabel('Predicted PE')
plt.title('Distribution of Predicted PE for Missing Values')
plt.grid(True)
plt.show()


Test MSE: 0.4124, R2: 0.5541


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

Wells with missing PE: [0 2 6]


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1️⃣ Plot predicted vs actual for test set
plt.figure(figsize=(8,6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)  # 1:1 line
plt.xlabel('Actual PE')
plt.ylabel('Predicted PE')
plt.title('Predicted vs Actual PE (Test Set)')
plt.grid(True)
plt.show()

# 2️⃣ Optionally, plot predicted PE for previously missing values
plt.figure(figsize=(10,6))
sns.histplot(missing_pe_data['PE'], kde=True, bins=30, color='skyblue')
plt.xlabel('Predicted PE')
plt.title('Distribution of Predicted PE for Missing Values')
plt.grid(True)
plt.show()


NameError: name 'y_test' is not defined

<Figure size 800x600 with 0 Axes>